# Join Statements

## Introduction

In this lab, you'll practice your knowledge on Join statements.

## Objectives

You will be able to:
- Write queries that make use of various types of Joins
- Join tables using foreign keys

## CRM Schema

In almost all cases, rather then just working with a single table we will typically need data from multiple tables. 
Doing this requires the use of **joins ** using shared columns from the two tables. 

In this lab, we'll use the same Customer Relationship Management (CRM) database we used in our lecture before!
<img src='Database-Schema.png' width=550>

## Connecting to the Database
Import the necessary packages and connect to the database **data.sqlite**.

In [2]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data.sqlite', detect_types=sqlite3.PARSE_COLNAMES)
cur = conn.cursor()


## Display the names of all the employees in Boston.

In [18]:
cur.execute('''select * from employees e
                        join offices o
                        on e.officeCode = o.officeCode
                        limit 20;
                        ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.tail(5)

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
15,1501,Bott,Larry,x2311,lbott@classicmodelcars.com,7,1102,Sales Rep,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
16,1504,Jones,Barry,x102,bjones@classicmodelcars.com,7,1102,Sales Rep,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA
17,1611,Fixter,Andy,x101,afixter@classicmodelcars.com,6,1088,Sales Rep,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
18,1612,Marsh,Peter,x102,pmarsh@classicmodelcars.com,6,1088,Sales Rep,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
19,1619,King,Tom,x103,tking@classicmodelcars.com,6,1088,Sales Rep,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC


In [13]:
cur.execute('''select firstName, lastName from employees join offices using (officeCode) where city = "Boston"''').fetchall()

[('Julie', 'Firrelli'), ('Steve', 'Patterson')]

## Do any offices have no employees?

In [26]:
cur.execute('''SELECT city, 
                    count(*) 
                    from offices 
                    left join employees 
                    using (officeCode) 
                    group by 1;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,city,count(*)
0,Boston,2
1,London,2
2,NYC,2
3,Paris,5
4,San Francisco,6


In [27]:
cur.execute('''SELECT employeeNumber from employees WHERE lastName IS NULL;''').fetchall()

[]

## Write 3 Questions of your own and answer them

In [108]:
# How many employees are there?
cur.execute('''select firstName, employeeNumber from employees;''').fetchall()

[('Diane', '1002'),
 ('Mary', '1056'),
 ('Jeff', '1076'),
 ('William', '1088'),
 ('Gerard', '1102'),
 ('Anthony', '1143'),
 ('Leslie', '1165'),
 ('Leslie', '1166'),
 ('Julie', '1188'),
 ('Steve', '1216'),
 ('Foon Yue', '1286'),
 ('George', '1323'),
 ('Loui', '1337'),
 ('Gerard', '1370'),
 ('Pamela', '1401'),
 ('Larry', '1501'),
 ('Barry', '1504'),
 ('Andy', '1611'),
 ('Peter', '1612'),
 ('Tom', '1619'),
 ('Mami', '1621'),
 ('Yoshimi', '1625'),
 ('Martin', '1702')]

In [30]:
#Top 5 customers in terms of purchases (i.e., payments made)?
cur.execute('''select customerName, amount 
                from payments
                left join customers
                using (customerNumber)
                order by 2
                asc; ''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,customerName,amount
0,Mini Gifts Distributors Ltd.,101244.59
1,Online Diecast Creations Co.,10223.83
2,"Blauer See Auto, Co.",10549.01
3,"Dragon Souveniers, Ltd.",105743.00
4,Mini Gifts Distributors Ltd.,11044.30


In [76]:
# Which employees manage more than 5 customers?
cur.execute('''select firstName, lastName, 
                count(salesRepEmployeeNumber) customersManaged
                from customers c
                left join employees e
                on c.salesRepEmployeeNumber = e.employeeNumber
                where firstName != "None"
                group by 1
                having count(salesRepEmployeeNumber) > 5
                order by customersManaged asc;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
df.head()

,firstName,lastName,customersManaged
0,Julie,Firrelli,6
1,Loui,Bondur,6
2,Martin,Gerard,6
3,Steve,Patterson,6
4,Foon Yue,Tseng,7


In [99]:
# Which customers does each employee manage?
cur.execute('''select firstName, lastName, 
                customerName
                from customers c
                left join employees e
                on c.salesRepEmployeeNumber = e.employeeNumber
                where firstName != "None"
                group by 3
                order by 3 asc;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head(10)

100


,firstName,lastName,customerName
0,Larry,Bott,"AV Stores, Co."
1,Gerard,Hernandez,Alpha Cognac
2,Foon Yue,Tseng,American Souvenirs Inc
3,Pamela,Castillo,Amica Models & Co.
4,Andy,Fixter,"Anna's Decorations, Ltd"
5,Gerard,Hernandez,Atelier graphique
6,Andy,Fixter,"Australian Collectables, Ltd"
7,Andy,Fixter,"Australian Collectors, Co."
8,Andy,Fixter,"Australian Gift Network, Co"
9,Gerard,Hernandez,Auto Associés & Cie.


## Level Up: Display the names of each product each employee has sold.

In [112]:
# Your code here
cur.execute('''select firstName, lastName, productName
                from employees e
                join customers c
                on e.employeeNumber = c.salesRepEmployeeNumber
                join 
                orders using(customerNumber)
                join 
                orderdetails using(orderNumber)
                join  
                products using(productCode)
                order by 1 asc;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head()
#df.firstName.unique()


2996


,firstName,lastName,productName
0,Andy,Fixter,1996 Moto Guzzi 1100i
1,Andy,Fixter,2003 Harley-Davidson Eagle Drag Bike
2,Andy,Fixter,P-51-D Mustang
3,Andy,Fixter,1936 Harley Davidson El Knucklehead
4,Andy,Fixter,1997 BMW R 1100 S


In [113]:
df.groupby(['firstName', 'lastName']).count()

productName
firstName lastName              
Andy      Fixter             185
Barry     Jones              220
Foon Yue  Tseng              142
George    Vanauf             211
Gerard    Hernandez          396
Julie     Firrelli           124
Larry     Bott               236
Leslie    Jennings           331
          Thompson           114
Loui      Bondur             177
Mami      Nishi              137
Martin    Gerard             114
Pamela    Castillo           272
Peter     Marsh              185
Steve     Patterson          152

## Level Up: Display the Number of Products each Employee Has sold

In [102]:
cur.execute('''select firstName, lastName, count(productName) num_sold
                from employees e
                join customers c
                on e.employeeNumber = c.salesRepEmployeeNumber
                join 
                orders using(customerNumber)
                join
                orderdetails using(orderNumber)
                join 
                products using(productCode)
                group by 1
                order by 1 asc;''')
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
df.head(15)

14


,firstName,lastName,num_sold
0,Andy,Fixter,185
1,Barry,Jones,220
2,Foon Yue,Tseng,142
3,George,Vanauf,211
4,Gerard,Hernandez,396
5,Julie,Firrelli,124
6,Larry,Bott,236
7,Leslie,Thompson,445
8,Loui,Bondur,177
9,Mami,Nishi,137


## Summary

Congrats! You now know how to use Join statements, along with leveraging your foreign keys knowledge!